In [ ]:
N = 1500
Q_BPR = 0.015
COEF_BPR = 3
PREPROC = 'minmax'
NORM_ALL = False

In [2]:
import numpy as np
import pandas as pd
from scipy.sparse import csr_matrix
from sklearn.preprocessing import minmax_scale, scale
from tqdm import tqdm
import time
import implicit

In [2]:
train_path = r"C:\Users\dlbol\Downloads\likes\likes\likes_data\train"
test_path = r"C:\Users\dlbol\Downloads\likes\likes\likes_data\test"
pred_path = r"C:\Users\dlbol\Downloads\likes\likes\likes_data\impicit_mix_als_bpr_"
score_path = r"C:\Users\dlbol\Downloads\likes\likes\likes_data\impicit_mix_als_bpr_ _score"

In [3]:
%%time

list_coord = []
idx = 0

with open(train_path) as f:
    lines = f.readlines()
    for line in tqdm(lines):
        tracks = line.strip().split(' ')
        for track in tracks:
            list_coord.append((idx, track))
        idx += 1
        
first_id_test = idx

with open(test_path) as f:
    lines = f.readlines()
    for line in tqdm(lines):
        tracks = line.strip().split(' ')
        for track in tracks:
            list_coord.append((idx, track))
        idx += 1
        
last_id_test = idx - 1

In [ ]:
first_id_test, last_id_test

In [5]:
%%time

rows = [i[0] for i in list_coord]
cols = [i[1] for i in list_coord]
arr_rows = np.array(rows)
arr_cols = np.array(cols)
arr_cols = arr_cols.astype(np.int32)
ones = np.ones(len(list_coord))
user_item = csr_matrix((ones, (arr_rows, arr_cols)), dtype=np.int32)

CPU times: total: 453 ms
Wall time: 458 ms


In [21]:
model_als = implicit.als.AlternatingLeastSquares(factors=200, 
                                                 regularization=10,
                                                 alpha=10,
                                                 use_native=True,
                                                 use_cg=False,
                                                 iterations=15,
                                                 random_state=42)
print('model_als.fit:')
model_als.fit(user_item)

model_bpr = implicit.bpr.BayesianPersonalizedRanking(factors=400, 
                                                     learning_rate=0.1,
                                                     regularization=0.01,
                                                     iterations=950,
                                                     verify_negative_samples=True,
                                                     random_state=42)   
print('model_als.fit:')
model_bpr.fit(user_item)

In [19]:
def N_opt(model_als, model_bpr, N=100, q_bpr=1):
    
    N_als = N
    N_bpr = int(N * q_bpr)
    
    userid = np.arange(last_id_test + 1)
        
    start_time = time.time()
    rec_als = model_als.recommend(userid, user_item, N=N_als, filter_already_liked_items=True)
    rec_als = rec_als[:, first_id_test:]
    print("\n\n--- %s seconds model_als ---" % (time.time() - start_time))

    start_time = time.time()
    rec_bpr = model_bpr.recommend(userid, user_item, N=N_bpr, filter_already_liked_items=True)
    rec_bpr = rec_bpr[:, first_id_test:]
    print("\n--- %s seconds model_bpr ---" % (time.time() - start_time))
    
    return rec_als, rec_bpr, N_als, N_bpr

In [20]:
def optimize(rec_als,
             rec_bpr,
             N_als,
             N_bpr,
             coef_bpr=1,
             norm_all=False,
             preproc=None):
    
    if not isinstance(norm_all, bool) or preproc not in ['minmax', 'standart', None]:
        raise ValueError('wrong params')
        
    if preproc is None:
        als_score = rec_als[1]
        bpr_score = rec_bpr[1]
    else:
        if norm_all:
            if preproc == 'minmax':
                als_score = minmax_scale(rec_als[1].flatten(), feature_range=(0, 1), axis=0).reshape((-1, N_als))
                bpr_score = minmax_scale(rec_bpr[1].flatten(), feature_range=(0, 1), axis=0).reshape((-1, N_bpr))
            if preproc == 'standart':
                als_score = scale(rec_als[1].flatten(), with_mean=True, with_std=True, axis=0).reshape((-1, N_als))
                bpr_score = scale(rec_bpr[1].flatten(), with_mean=True, with_std=True, axis=0).reshape((-1, N_bpr))    
        else:
            if preproc == 'minmax':
                als_score = minmax_scale(rec_als[1], feature_range=(0, 1), axis=1)
                bpr_score = minmax_scale(rec_bpr[1], feature_range=(0, 1), axis=1)   
            if preproc == 'standart':
                als_score = scale(rec_als[1], with_mean=True, with_std=True, axis=1)
                bpr_score = scale(rec_bpr[1], with_mean=True, with_std=True, axis=1)     

    result = []
    scores = []
    for i in tqdm(range(rec_als[0].shape[0])):
        als_df = pd.DataFrame({'trak_id': rec_als[0][i], 'als_score': als_score[i]})
        bpr_df = pd.DataFrame({'trak_id': rec_bpr[0][i], 'bpr_score': bpr_score[i] * coef_bpr})
        df = pd.merge(als_df, bpr_df, how="outer", on='trak_id', sort=False)
        df['als_prior'] = df.als_score.where(~df.als_score.isna(), df.bpr_score)
        df['total_score'] = np.where((df.als_score.isna()) | (df.bpr_score.isna()), df.als_prior, 
                                     (df.als_score + df.bpr_score) / 2)
        final_rec = df.sort_values(by='total_score', ascending=False).trak_id.values[:100]
        result.append(' '.join(map(str, final_rec)) + '\n')
        final_scr = df.sort_values(by='total_score', ascending=False).total_score.values[:100]
        result.append(' '.join(map(str, final_scr)) + '\n')
        
        
    with open(pred_path, 'w') as f:
        f.writelines(result)

    with open(score_path, 'w') as f:
        f.writelines(scores)

In [22]:
rec_als, rec_bpr, N_als, N_bpr = N_opt(model_als, model_bpr, N=N, q_bpr=Q_BPR)

In [ ]:
optimize(rec_als,
         rec_bpr,
         N_als,
         N_bpr,
         coef_bpr=COEF_BPR,
         norm_all=NORM_ALL,
         preproc=PREPROC)